# Import libraries

In [12]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

# Load data

In [13]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "is_healthy.csv"), index_col=[0, 1])
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283245        0.106067                     0.106067   
         SAMN03283294        0.000000                     0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283245                                            0.106067   
         SAMN03283294                                            0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283245                                               0.106067                      
         SAMN03283294                                               0.000000                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283245                                               0.106067                                             
         SAMN03283294                                               0.000000                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283245                                               0.072453                                                                   
         SAMN03283294                                               0.000000                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283245                              True
         SAMN03283294                              True

# Generate predictions in 10-fold cross validation! (repeated 10 times)

In [14]:
num_repeats = 10
n_splits = 10
kfold = KFold(n_splits=n_splits, shuffle=True)

logit_prediction_list = []

for i in tqdm(range(num_repeats)):
    # instantiate the logistic regression classifier (gmhi2)
    gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

    # generate logit predictions in 10 fold cross validation
    GMHI2_scores_cv = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
        cv=kfold, verbose=0, n_jobs=-1
    )

    GMHI2_scores_cv = pd.DataFrame(GMHI2_scores_cv, index=y.index, columns=["GMHI2_cv"])
    logit_prediction_list.append(GMHI2_scores_cv)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:25<00:00,  2.54s/it]


In [30]:
cutoff = 1
balanced_accuracies = [balanced_accuracy_score(y[(abs(lo) >= cutoff).values], lo[(abs(lo) >= cutoff).values] > 0) for lo in logit_prediction_list]
balanced_accuracies, np.mean(balanced_accuracies)

([0.8949656796873977,
  0.8970690142751967,
  0.8939807604725006,
  0.8912126455521361,
  0.8957366389309354,
  0.8952594933040755,
  0.8935649163859423,
  0.8930716483073418,
  0.8947432738701602,
  0.8958010605101518],
 0.8945405131295837)

In [31]:
mat_sum = np.zeros([2 , 2])

for lo in logit_prediction_list:
    idx = (abs(lo) >= cutoff).values
    y_curr = y[idx]
    lo_curr = lo[idx]
    mat = confusion_matrix(y_curr, lo_curr > 0)
    mat_sum += mat

In [32]:
tn, fp, fn, tp = mat_sum.ravel()
mat_sum

array([[13985.,  1536.],
       [ 3630., 28792.]])

In [33]:
pd.DataFrame(mat_sum, columns=["Predicted Nonhealthy", "Predicted Healthy"], index=["Actual Nonhealthy", "Actual healthy"])

,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,13985.0,1536.0
Actual healthy,3630.0,28792.0


In [34]:
mat_sum.sum(), mat_sum.sum() / 88690

(47943.0, 0.540568271507498)